# Example QCoDeS Sweep Script for Dilution Refrigerator

## This block of code must be run- it imports all the relevant libraries and packages

In [1]:
from threaded_sweep import Sweep1D, Sweep0D, Sweep2D
from daq_driver import Daq, DaqAOChannel, DaqAIChannel
import nidaqmx
import time
import numpy as np
import qcodes as qc
from util import init_database
from tracking import *
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.database import initialise_or_create_database_at
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

## The next blocks are specific to various instruments - you only need to run the blocks for the instruments you wish to control/get data from.

### If you want to use the Lakeshore temperature controller, run this to initialize it

In [ ]:
ls = Model_372('lakeshore_372', 'TCPIP::10.155.95.127::7777::SOCKET')
ls.ch01.enabled(True)
ls.ch02.enabled(True)
ls.ch03.enabled(True)
ls.ch05.enabled(True)
ls.ch06.enabled(True)
for ch in ls.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

### Initialize the magnet here

In [ ]:
magnet = AMI430("Magnet", address="10.155.94.127", port=7180, has_current_rating=False)

### Initialize the DAQ here

In [2]:
daq = Daq("Dev1", "Daq")

## Optional: change your parameter labels to more descriptive values for clarity

In [ ]:
# Command takes the form
# <QCoDeS Parameter>.label="label descriptor"
daq.ai0.voltage.label="Isd"
daq.ai1.voltage.label="Vxx"
daq.ai2.voltage.label="Vxy"

## Create your sweep object here! 
### The arguments for the 1D sweep shown here are, in order, the parameter to be swept, the starting point, ending point, step value, delay between steps, flag to sweep back and forth, whether to save data, and plot data, and whether to plot the parameters against time or the sweeping parameter
### The only required arguments are the parameter, start, stop, and step values

In [4]:
sweep = Sweep1D(daq.ao0.voltage, 0, 2, 0.05, inter_delay = 0.01, bidirectional = False, save_data = True, 
            plot_data = True, x_axis_time = 1)

## Tell the sweep which other parameters to track during the sweep. Here, we have added the magnetic field, the temperature, and a couple DAQ channels

In [5]:
sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage)

## Now, we initialize/create the database we are saving to (if you want to save data)
## You can run this code between each run to give the data identifiers through *exp_name* and *sample_name*

In [3]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "temp.db"
    exp_name = "Example experiment- Magnetic field dependence"
    sample_name = "Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others"
    init_database(database_name, exp_name, sample_name, sweep)
except:
    print("Error opening database")

## Now, we can start the sweep! This will automatically start data collection, plotting, saving, etc

In [7]:
%matplotlib qt
sweep.start()

Ramping to our starting setpoint value of 0 V
Sweeping ao0 Voltage to 0 V
Ramping ao0 Voltage to 0 . . . 


C:\Users\erunb\Anaconda3\envs\qcodes\lib\site-packages\matplotlib\figure.py:2366: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


Done with the sweep, ao0 Voltage=-3.191891195797325e-16
Done ramping ao0 Voltage to 0
Sweeping ao0 Voltage to 2 V
Starting experimental run with id: 17
Done with the sweep, ao0 Voltage=2.000000000000001


### To stop the sweep, run the stop command below. To restart it, you can run the start command from above again.

In [ ]:
sweep.stop()